In [1]:
#cell-width control
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Imports

In [2]:
#packages
import numpy
import tensorflow as tf
from tensorflow.core.example import example_pb2

#utils
import os
import random
import pickle
import struct
import time
from generators import *

#keras
import keras
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Model, Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout, Activation, Concatenate, Dot, Embedding, LSTM, Conv1D, MaxPooling1D, Input, Lambda
    #callbacks
from keras.callbacks import TensorBoard, ModelCheckpoint, Callback


Using TensorFlow backend.


# Seed

In [3]:
sd = 3
from numpy.random import seed
seed(sd)
from tensorflow import set_random_seed
set_random_seed(sd)

# CPU usage

In [4]:
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

# Global parameters

In [5]:
# Embedding
max_features = 400000
maxlen_text = 400
maxlen_summ = 80
embedding_size = 100 #128

# Convolution
kernel_size = 5
filters = 64
pool_size = 4

# LSTM
lstm_output_size = 70

# Training
batch_size = 32
epochs = 20

# Load data

In [6]:
data_dir = '/mnt/disks/500gb/experimental-data-mini/experimental-data-mini/pseudorandom-dist-1to1/1to1/'
processing_dir = '/mnt/disks/500gb/stats-and-meta-data/400000/'

with open(data_dir+'partition.pickle', 'rb') as handle: partition = pickle.load(handle)
with open(data_dir+'labels.pickle', 'rb') as handle: labels = pickle.load(handle)

with open(processing_dir+'tokenizer.pickle', 'rb') as handle: tokenizer = pickle.load(handle)
embedding_matrix = numpy.load(processing_dir+'embedding_matrix.npy')

#the p_n constant
c = 80000

#stats
maxi = numpy.load(processing_dir+'training-stats-all/maxi.npy')
mini = numpy.load(processing_dir+'training-stats-all/mini.npy')
sample_info = (numpy.random.uniform, mini,maxi)

# Model

In [7]:
#2way input
text_input = Input(shape=(maxlen_text,embedding_size), dtype='float32')
summ_input = Input(shape=(maxlen_summ,embedding_size), dtype='float32')

#1way dropout
#text_route = Dropout(0.25)(text_input)
summ_route = Dropout(0.25)(summ_input)

#1way conv
#text_route = Conv1D(filters,
                 #kernel_size,
                 #padding='valid',
                 #activation='relu',
                 #strides=1)(text_route)
summ_route = Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1)(summ_route)

#1way max pool
#text_route = MaxPooling1D(pool_size=pool_size)(text_route)
summ_route = MaxPooling1D(pool_size=pool_size)(summ_route)

#1way lstm
#text_route = LSTM(lstm_output_size)(text_route)
summ_route = LSTM(lstm_output_size)(summ_route)

#negate results
#merged = Lambda(lambda x: -1*x)(merged)

#add p_n constant
#merged = Lambda(lambda x: x + c)(merged)

#output
output = Dense(1, activation='sigmoid')(summ_route)

#define model
model = Model(inputs=[text_input, summ_input], outputs=[output])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [8]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 80, 100)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 80, 100)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 76, 64)            32064     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 19, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 70)                37800     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 71        
Total params: 69,935
Trainable params: 69,935
Non-trainable params: 0
_________________________________________________________________
None


# Train model

In [ ]:
#callbacks
class BatchHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.accs = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.accs.append(logs.get('acc'))
        
history = BatchHistory()
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=batch_size, write_graph=True, write_grads=True)
modelcheckpoint = ModelCheckpoint('best.h5', monitor='val_loss', verbose=0, save_best_only=True, mode='min', period=1)

#batch generator parameters
params = {'dim': [(maxlen_text,embedding_size),(maxlen_summ,embedding_size)],
          'batch_size': batch_size,
          'shuffle': True,
         'tokenizer':tokenizer,
         'embedding_matrix':embedding_matrix,
         'maxlen_text':maxlen_text,
         'maxlen_summ':maxlen_summ,
         'data_dir':data_dir,
         'sample_info':sample_info}

#generators
training_generator = ContAllGenerator(partition['train'], labels, **params)
validation_generator = ContAllGenerator(partition['validation'], labels, **params)

# Train model on dataset
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=5,
                   epochs=epochs,
                   callbacks=[tensorboard, modelcheckpoint, history])

Epoch 1/20
17944/17944 [==============================] - 1973s 110ms/step - loss: 0.4838 - acc: 0.7618 - val_loss: 0.4538 - val_acc: 0.7855
Epoch 2/20
17944/17944 [==============================] - 1929s 108ms/step - loss: 0.4434 - acc: 0.7926 - val_loss: 0.4457 - val_acc: 0.7913
Epoch 3/20
17944/17944 [==============================] - 1968s 110ms/step - loss: 0.4355 - acc: 0.7979 - val_loss: 0.4388 - val_acc: 0.7959
Epoch 4/20
17944/17944 [==============================] - 1991s 111ms/step - loss: 0.4313 - acc: 0.8001 - val_loss: 0.4345 - val_acc: 0.7982
Epoch 5/20
17944/17944 [==============================] - 1899s 106ms/step - loss: 0.4288 - acc: 0.8013 - val_loss: 0.4375 - val_acc: 0.7950
Epoch 6/20
17944/17944 [==============================] - 1906s 106ms/step - loss: 0.4265 - acc: 0.8035 - val_loss: 0.4293 - val_acc: 0.8014
Epoch 7/20
17944/17944 [==============================] - 1888s 105ms/step - loss: 0.4247 - acc: 0.8047 - val_loss: 0.4323 - val_acc: 0.8009
Epoch 8/20
17

In [ ]:
with open('losses.pickle', 'wb') as handle: pickle.dump(history.losses, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('accs.pickle', 'wb') as handle: pickle.dump(history.accs, handle, protocol=pickle.HIGHEST_PROTOCOL)